# Move EDA

This notebook is used to identify moves and how they are learned. For our simulation, we only want to include moves that are pokemon specific; not TM or HM based. The data also includes how it is learned in other versions of Pokemon - we only care about the Red and Blue versions.

In [1]:
from glob import glob
import json
import pandas as pd

In [2]:
def load_json(fp):
    data = None
    with open(fp) as f:
        data = json.load(f)
    
    return data

In [3]:
pokemon = []
for fp in glob('../data/raw/pokemon/*.json'):
    pokemon.append(load_json(fp))

In [4]:
def redblue_moves(pokemon):
    moves = []
    # find all red/blue moves
    for move in pokemon['moves']:
        for version_details in move['version_group_details']:
            game_version = version_details['version_group']['name']
            method = version_details['move_learn_method']['name']
            if game_version == 'red-blue' and method == 'level-up':
                moves.append({
                    'pokemon': pokemon['name'],
                    'move_name': move['move']['name'],
                    'move_url': move['move']['url']
                })
    return moves

In [5]:
pokemon_moves = []

for character in pokemon:
    pokemon_moves = pokemon_moves + redblue_moves(character)

In [6]:
pokemon_moves_df = pd.DataFrame(pokemon_moves)

In [7]:
pokemon_moves_df.head(20)

,move_name,move_url,pokemon
0,swords-dance,https://pokeapi.co/api/v2/move/14/,farfetchd
1,sand-attack,https://pokeapi.co/api/v2/move/28/,farfetchd
2,fury-attack,https://pokeapi.co/api/v2/move/31/,farfetchd
3,leer,https://pokeapi.co/api/v2/move/43/,farfetchd
4,peck,https://pokeapi.co/api/v2/move/64/,farfetchd
5,agility,https://pokeapi.co/api/v2/move/97/,farfetchd
6,slash,https://pokeapi.co/api/v2/move/163/,farfetchd
7,stomp,https://pokeapi.co/api/v2/move/23/,rhyhorn
8,horn-attack,https://pokeapi.co/api/v2/move/30/,rhyhorn
9,fury-attack,https://pokeapi.co/api/v2/move/31/,rhyhorn


# Merge Move Stats with Pokemon Moves

In [8]:
unique_moves = pokemon_moves_df['move_name'].unique()

In [9]:
move_details = []

for move in unique_moves:
    fp = '../data/raw/moves/{}.json'.format(move)
    move_details.append(load_json(fp))

In [10]:
def get_move_stats(move):
    meta = move.get('meta')
    
    # find effect english
    effect = {}
    for entry in move['effect_entries']:
        if entry['language']['name'] == 'en':
            effect = entry
    
    # find stat change
    stat_change = move.get('stat_changes')
    if stat_change:
        stat_change = stat_change[-1]
    else:
        stat_change = {'stat': {}}
        
    
    return {
        'move_name': move.get('name'),
        'move_accuracy': move.get('accuracy'),
        'move_crit_rate': meta.get('crit_rate'),
        'move_drain': meta.get('drain'),
        'move_flinch_chance': meta.get('flinch_chance'),
        'move_healing': meta.get('healing'),
        'move_max_hits': meta.get('max_hits'),
        'move_max_turns': meta.get('max_turns'),
        'move_min_hits': meta.get('min_hits'),
        'move_min_turns': meta.get('min_turns'),
        'move_stat_chance': meta.get('stat_chance'),
        'move_power': move.get('power'),
        'move_pp': move.get('pp'),
        'move_priority': move.get('priority'),
        'move_type': move.get('type').get('name'),
        'move_effect_chance': move.get('effect_chance'),
        'move_effect': effect.get('effect'),
        'move_stat_change': stat_change.get('stat').get('name'),
        'move_stat_change_amount': stat_change.get('change'),
        'move_category': move.get('meta').get('category').get('name'),
        'move_target': move.get('target').get('name'),
        'move_damage_class': move.get('damage_class').get('name')
    }

In [11]:
move_stats = []

for move in move_details:
    move_stats.append(get_move_stats(move))

In [12]:
move_stats_df = pd.DataFrame(move_stats)

In [13]:
move_stats_df.head()

,move_accuracy,move_category,move_crit_rate,move_damage_class,move_drain,move_effect,move_effect_chance,move_flinch_chance,move_healing,move_max_hits,...,move_min_turns,move_name,move_power,move_pp,move_priority,move_stat_chance,move_stat_change,move_stat_change_amount,move_target,move_type
0,NaN,net-good-stats,0,status,0,Raises the user's Attack by two stages.,NaN,0,0,NaN,...,NaN,swords-dance,NaN,20,0,0,attack,2.0,user,normal
1,100.0,net-good-stats,0,status,0,Lowers the target's accuracy by one stage.,NaN,0,0,NaN,...,NaN,sand-attack,NaN,15,0,0,accuracy,-1.0,selected-pokemon,ground
2,85.0,damage,0,physical,0,Inflicts regular damage. Hits 2–5 times in on...,NaN,0,0,5.0,...,NaN,fury-attack,15.0,20,0,0,None,NaN,selected-pokemon,normal
3,100.0,net-good-stats,0,status,0,Lowers the target's Defense by one stage.,100.0,0,0,NaN,...,NaN,leer,NaN,30,0,100,defense,-1.0,all-opponents,normal
4,100.0,damage,0,physical,0,Inflicts regular damage.,NaN,0,0,NaN,...,NaN,peck,35.0,35,0,0,None,NaN,selected-pokemon,flying


In [14]:
move_stats_df['move_stat_change'].unique()

array(['attack', 'accuracy', None, 'defense', 'speed', 'special-attack',
       'special-defense', 'evasion'], dtype=object)

In [15]:
len(pokemon_moves_df)

1090

In [16]:
len(move_stats_df)

145

In [17]:
len(pd.merge(pokemon_moves_df, move_stats_df, how='inner', on='move_name'))

1090

In [18]:
pokemon_moves_detailed_df = pd.merge(pokemon_moves_df, move_stats_df, how='inner', on='move_name')

In [19]:
pokemon_moves_detailed_df.to_csv('../data/processed/pokemon_moves_detailed.csv', index=False)